In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pylab as plt
from scipy.interpolate import interp1d
from scipy.optimize import least_squares
import matplotlib.ticker as ticker
from scipy.integrate import quad

import sys

sys.path.append("..")

from analytical_option_formulae.option_types.vanilla_option import VanillaOption

vanilla_option = VanillaOption()

# please adjust this before running, its either SPX or SPY
filename = "SPX_options"
part3_date = pd.to_datetime("20210115", format="%Y%m%d")

In [2]:
# implied volatility reporting


def implied_volatility(
    S: float, K: float, r: float, price: float, T: float, options_type: str
) -> float:
    try:
        bs_model = lambda x: vanilla_option.black_scholes_model(S, K, r, x, T)
        if options_type.lower() == "call":
            implied_vol = brentq(
                lambda x: price - bs_model(x).calculate_call_price(), 1e-12, 10.0
            )
        elif options_type.lower() == "put":
            implied_vol = brentq(
                lambda x: price - bs_model(x).calculate_put_price(), 1e-12, 10.0
            )
        else:
            raise NameError("Payoff type not recognized")
    except Exception:
        implied_vol = np.nan

    return implied_vol

In [3]:
df = pd.read_csv(f"../data/{filename}.csv")
df["mid"] = 0.5 * (df["best_bid"] + df["best_offer"])
df["strike"] = df["strike_price"] * 0.001
df["payoff"] = df["cp_flag"].map(lambda x: "call" if x == "C" else "put")
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
df["exdate"] = pd.to_datetime(df["exdate"], format="%Y%m%d")
df["days_to_expiry"] = (df["exdate"] - df["date"]).dt.days
df["years_to_expiry"] = df["days_to_expiry"] / 365
df = df[df["exdate"] == part3_date]
df = df.reset_index(drop=True)

# setup rates calculator
rates_df = pd.read_csv("../data/zero_rates_20201201.csv")
rate_interpolate = interp1d(rates_df["days"], rates_df["rate"])
df["rates"] = (
    rate_interpolate(df["days_to_expiry"]) / 100.0
)  # make it in fractions so i dont forget

try:
    if filename.lower() == "spy_options":
        S = 366.02
    elif filename.lower() == "spx_options":
        S = 3662.45
    else:
        raise NameError("unknown input file")
except Exception as e:
    print(e)

# impl market volatility column
df["vols"] = df.apply(
    lambda x: implied_volatility(
        S, x["strike"], x["rates"], x["mid"], x["years_to_expiry"], x["payoff"]
    ),
    axis=1,
)
df.dropna(inplace=True)

df

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid,strike,payoff,days_to_expiry,years_to_expiry,rates,vols
91,2020-12-01,2021-01-15,C,2570000,1087.2,1099.6,E,1093.40,2570.0,call,45,0.123288,0.002051,0.361156
92,2020-12-01,2021-01-15,C,2575000,1082.3,1094.6,E,1088.45,2575.0,call,45,0.123288,0.002051,0.364647
93,2020-12-01,2021-01-15,C,2580000,1077.3,1089.6,E,1083.45,2580.0,call,45,0.123288,0.002051,0.362672
94,2020-12-01,2021-01-15,C,2590000,1067.4,1079.7,E,1073.55,2590.0,call,45,0.123288,0.002051,0.368049
96,2020-12-01,2021-01-15,C,2610000,1047.6,1059.9,E,1053.75,2610.0,call,45,0.123288,0.002051,0.374739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,2020-12-01,2021-01-15,P,5000000,1330.8,1346.6,E,1338.70,5000.0,put,45,0.123288,0.002051,0.391899
734,2020-12-01,2021-01-15,P,5100000,1428.3,1451.2,E,1439.75,5100.0,put,45,0.123288,0.002051,0.434093
735,2020-12-01,2021-01-15,P,5200000,1528.3,1551.2,E,1539.75,5200.0,put,45,0.123288,0.002051,0.455783
736,2020-12-01,2021-01-15,P,5300000,1628.2,1651.1,E,1639.65,5300.0,put,45,0.123288,0.002051,0.475063


In [4]:
# create market DF for each timestamp
call_df = df[df["payoff"] == "call"]
put_df = df[df["payoff"] == "put"]
strikes = sorted(df["strike"].unique())
impliedvols = []
option_type = []
option_price = []
for K in strikes:
    if K > S:
        impliedvols.append(call_df[call_df["strike"] == K]["vols"].values[0])
        option_type.append("call")
        option_price.append(call_df[call_df["strike"] == K]["mid"].values[0])
    else:
        impliedvols.append(put_df[put_df["strike"] == K]["vols"].values[0])
        option_type.append("put")
        option_price.append(put_df[put_df["strike"] == K]["mid"].values[0])

day_market_df = pd.DataFrame(
    {
        "strike": strikes,
        "vol": impliedvols,
        "option_type": option_type,
        "option_price": option_price,
    }
)
day_market_df[
    (day_market_df["strike"] > (S - 15)) & (day_market_df["strike"] < (S + 15))
]

,strike,vol,option_type,option_price
285,3650.0,0.190902,put,91.20
286,3655.0,0.189871,put,93.15
287,3660.0,0.188860,put,95.15
288,3665.0,0.180798,call,91.95
289,3670.0,0.179825,call,89.05
290,3675.0,0.178963,call,86.25


In [5]:
start_date = pd.to_datetime("2020-12-01")
end_date = pd.to_datetime("2021-01-15")
day_diff = (end_date - start_date).days
T = day_diff / 365.0
rate = rate_interpolate(day_diff) / 100.0

# use interpolate or closest value ?
sigma_interp = np.interp(S, day_market_df["strike"], day_market_df["vol"])
sigma_interp

near_atm_put = day_market_df[day_market_df["option_type"] == "put"].iloc[-1]
near_atm_call = day_market_df[day_market_df["option_type"] == "call"].iloc[0]
approx_option = (
    near_atm_call
    if near_atm_put["strike"] - S > near_atm_call["strike"] - S
    else near_atm_put
)
sigma_approx = implied_volatility(
    S, S, rate, approx_option["option_price"], T, approx_option["option_type"]
)
# approx_option, i will continue using sigma_interp, but
print(
    f"BS sigma using interpolation : {sigma_interp} , BS sigma using approx to market data : {sigma_approx}"
)

# compare output data using call price
call_price_with_sigma_interp = vanilla_option.black_scholes_model(
    S, S, rate, sigma_interp, T
).calculate_call_price()
call_price_with_sigma_approx = vanilla_option.black_scholes_model(
    S, S, rate, sigma_approx, T
).calculate_call_price()
print(
    f"BS Call using sigma interp : {call_price_with_sigma_interp} , BS Call using sigma approx : {call_price_with_sigma_approx}"
)


# conclusion : they are close enough, i am more comfortable with interp though
bs_sigma = sigma_interp

BS sigma using interpolation : 0.1849096526276905 , BS sigma using approx to market data : 0.18642518487248805
BS Call using sigma interp : 95.2990207952223 , BS Call using sigma approx : 96.07601517378384


In [6]:
# bachelier sigma we will use formula straight away
def bach_sigma(v_call, rate, T):
    return (v_call * np.exp(rate * T)) / (np.sqrt(T / 2 * np.pi))


# what price to use for bach sigma? again, lets go back to approxes
# 1. using the approx price BS call_price_with_sigma_interp
bach_sigma_interp = bach_sigma(call_price_with_sigma_interp, rate, T)

# 2. using market data, seems need to use the 'CALL' price
bach_sigma_approx = bach_sigma(near_atm_call["option_price"], rate, T)

print(
    f"Bachelier Sigma using Interpolation Price = {bach_sigma_interp}, Bachelier Sigma using Approx Call Price  = {bach_sigma_approx}"
)

# conclusion : they are close enough, i am more comfortable with interp though
bach_sigma = bach_sigma_interp

Bachelier Sigma using Interpolation Price = 216.60999689666605, Bachelier Sigma using Approx Call Price  = 208.99783700240258


# Exotic No.1 : Black-Scholes

Expected Black Scholes payoff is defined as

$$
E[V_T]= {S_0}^\frac{1}{3}e^{\frac{rT}{3}}e^{\frac{-\sigma^2T}{9}} + 1.5(log{S_0} + (r-\frac{\sigma^2}{2})T) + 10
$$

Therefore, the price is

$$
V_0 = e^{-rT}E[V_T]
$$


In [7]:
# black scholes payoff
def bs_price(S, rate, sigma, T):
    return np.exp(-rate * T) * (
        (
            np.power(S, 1.0 / 3.0)
            * np.exp((rate - 0.5 * sigma**2) * T * (1.0 / 3.0))
            * np.exp(0.5 * (1.0 / 9.0) * T * sigma**2)
            + 1.5 * (np.log(S) + (rate - 0.5 * sigma**2) * T)
            + 10
        )
    )

# Exotic No.1 : Bachelier

Expected Bachelier payoff defined as

$$
E[V_T] = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} (S_0 + \sigma S_0 x)^\frac{1}{3} e^\frac{-x^2}{2}\,dx +
\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} 1.5log(S_0 + \sigma S_0 x) e^\frac{-x^2}{2}\,dx
+10
$$

Note $$S_T = S_0 + \sigma S_0W_T$$

Therefore, the price is

$$
V_0 = e^{-rT}E[V_T]
$$


In [8]:
def integrand_1(x, S, sigma, T):
    return (
        (1 / np.sqrt(2 * np.pi))
        * np.power((S + sigma * S * np.sqrt(T) * x), 1.0 / 3.0)
        * np.exp(-0.5 * np.power(x, 2))
    )


def integrand_2(x, S, sigma, T):
    return (
        (1 / np.sqrt(2 * np.pi))
        * 1.5
        * np.log(S + sigma * S * np.sqrt(T) * x)
        * np.exp(-0.5 * np.power(x, 2))
    )


def bachelier_price(S, rate, sigma, T):
    lower_bound = -1 / (sigma * np.sqrt(T))  # log term lower bound
    I_1 = quad(lambda x: integrand_1(x, S, sigma, T), lower_bound, np.inf)
    I_2 = quad(lambda x: integrand_2(x, S, sigma, T), lower_bound, np.inf)
    V_0_bachelier = np.exp(-rate * T) * (I_1[0] + I_2[0] + 10)
    return V_0_bachelier

# Exotic No.1 : SABR & Static Replication

For SABR payoff, we must retrieve the volatility using previous calibration result

$$
h(S_T) = S_T^{\frac{1}{3}} + 1.5 log(S_T) + 10 \\
h''(S_T) = -\frac{2}{9}S_T^{-\frac{5}{3}} - 1.5\frac{1}{S_T^2} \\
F = S_0 e ^ {rT}
$$

Therefore, the price is

$$
V_0 = e^{-rT}h(F) + \int_{0}^{F} h''(K)P(K) \,dK + \int_{F}^{\infty} h''(K)C(K) \,dK
$$


In [9]:
# SABR Related
spy_sabr_params = {
    17: {
        "alpha": 0.6654021927640178,
        "rho": -0.411899860796424,
        "nu": 5.249981412437772,
    },
    45: {
        "alpha": 0.9081326347161665,
        "rho": -0.48877944853880195,
        "nu": 2.7285163433392956,
    },
    80: {
        "alpha": 1.1209243554286754,
        "rho": -0.632939170293977,
        "nu": 1.742224770841638,
    },
}
spx_sabr_params = {
    17: {
        "alpha": 1.2122899548875727,
        "rho": -0.3009002722327193,
        "nu": 5.459761425517507,
    },
    45: {
        "alpha": 1.8165044343707681,
        "rho": -0.40430176121217326,
        "nu": 2.7901583189643917,
    },
    80: {
        "alpha": 2.140132609863112,
        "rho": -0.5749338828217627,
        "nu": 1.8417469754894316,
    },
}

try:
    if filename.lower() == "spy_options":
        alpha = spy_sabr_params[day_diff]["alpha"]
        rho = spy_sabr_params[day_diff]["rho"]
        nu = spy_sabr_params[day_diff]["nu"]
        beta = 0.7
    elif filename.lower() == "spx_options":
        alpha = spx_sabr_params[day_diff]["alpha"]
        rho = spx_sabr_params[day_diff]["rho"]
        nu = spx_sabr_params[day_diff]["nu"]
        beta = 0.7
    else:
        raise NameError("unknown input file")
except Exception as e:
    print(e)


# i just use prof Tee
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta) ** 2) / 24) * alpha * alpha / (F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        VolAtm = alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))
        sabrsigma = VolAtm
    else:
        z = (nu / alpha) * ((F * X) ** (0.5 * (1 - beta))) * np.log(F / X)
        zhi = np.log((((1 - 2 * rho * z + z * z) ** 0.5) + z - rho) / (1 - rho))
        numer1 = (((1 - beta) ** 2) / 24) * ((alpha * alpha) / ((F * X) ** (1 - beta)))
        numer2 = 0.25 * rho * beta * nu * alpha / ((F * X) ** ((1 - beta) / 2))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z
        denom1 = ((1 - beta) ** 2 / 24) * (np.log(F / X)) ** 2
        denom2 = (((1 - beta) ** 4) / 1920) * ((np.log(F / X)) ** 4)
        denom = ((F * X) ** ((1 - beta) / 2)) * (1 + denom1 + denom2) * zhi
        sabrsigma = numer / denom

    return sabrsigma

In [10]:
def h_func(F):
    return np.power(F, 1.0 / 3.0) + 1.5 * np.log(F) + 10


def h_second_deriv(F):
    return (-2 / 9) * np.power(F, -5.0 / 3.0) - 1.5 * np.power(F, -2)


def h_put_integ(x, S, rate, T):
    # self, S: float, K: float, r: float, sigma: float, T: float
    F = S * np.exp(rate * T)
    sigma_p = SABR(F, x, T, alpha, beta, rho, nu)
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma_p, T)
    return h_second_deriv(x) * bsmodel.calculate_put_price()


def h_call_integ(x, S, rate, T):
    F = S * np.exp(rate * T)
    sigma_c = SABR(F, x, T, alpha, beta, rho, nu)
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma_c, T)
    return h_second_deriv(x) * bsmodel.calculate_call_price()


def sabr_price(S, rate, T):
    F = S * np.exp(rate * T)
    I_put = quad(lambda x: h_put_integ(x, S, rate, T), 1e-6, F)
    I_call = quad(lambda x: h_call_integ(x, S, rate, T), F, np.inf)
    V_0_SABR = np.exp(-rate * T) * h_func(F) + I_put[0] + I_call[0]
    return V_0_SABR

In [11]:
V_0_black_scholes = bs_price(S, rate, bs_sigma, T)
V_0_bachelier = bachelier_price(S, rate, bach_sigma, T)
V_0_SABR = sabr_price(S, rate, T)

# Model Free Integrated Variance - Black Scholes

$$
E\Biggl[\int_{0}^{T} \sigma_t^2 \,dt\Biggr] = 2e^{rT}\Biggl(\int_{0}^{F} \frac{P(K)}{K^2} \,dK + \int_{F}^{\infty} \frac{C(K)}{K^2} \,dK\Biggr)
$$


In [12]:
def bs_put_integ(x, S, rate, sigma, T):
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma, T)
    return bsmodel.calculate_put_price() / (np.power(x, 2))


def bs_call_integ(x, S, rate, sigma, T):
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma, T)
    return bsmodel.calculate_call_price() / (np.power(x, 2))

In [13]:
F = S * np.exp(rate * T)
I_put = quad(lambda x: bs_put_integ(x, S, rate, bs_sigma, T), 0.0, F)
I_call = quad(lambda x: bs_call_integ(x, S, rate, bs_sigma, T), F, np.inf)
bs_E_var = 2 * np.exp(rate * T) * (I_put[0] + I_call[0])
bs_E_var

0.004215400228959317

# Model Free Integrated Variance - Bachelier


In [14]:
def bach_put_integ(x, S, rate, sigma, T):
    bachmodel = vanilla_option.bachelier_model(S, x, rate, sigma, T)
    return bachmodel.calculate_put_price() / (np.power(x, 2))


def bach_call_integ(x, S, rate, sigma, T):
    bachmodel = vanilla_option.bachelier_model(S, x, rate, sigma, T)
    return bachmodel.calculate_call_price() / (np.power(x, 2))

In [15]:
F = S * np.exp(rate * T)
I_put = quad(lambda x: bach_put_integ(x, S, rate, bach_sigma, T), 0.0, F)
I_call = quad(lambda x: bach_call_integ(x, S, rate, bach_sigma, T), F, np.inf)
bach_E_var = 2 * np.exp(rate * T) * (I_put[0] + I_call[0])
bach_E_var

C:\Users\kilo\AppData\Local\Temp\ipykernel_11520\3539066940.py:2: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  I_put = quad(lambda x: bach_put_integ(x, S, rate, bach_sigma, T), 0.0, F)


11.051429272300238

# Model Free Integrated Variance - SABR & Static Replication


In [16]:
def sabr_put_integ(x, S, rate, T, alpha, beta, rho, nu):
    F = S * np.exp(rate * T)
    sigma_p = SABR(F, x, T, alpha, beta, rho, nu)
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma_p, T)
    return bsmodel.calculate_put_price() / (np.power(x, 2))


def sabr_call_integ(x, S, rate, T, alpha, beta, rho, nu):
    F = S * np.exp(rate * T)
    sigma_c = SABR(F, x, T, alpha, beta, rho, nu)
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma_c, T)
    return bsmodel.calculate_call_price() / (np.power(x, 2))

In [17]:
F = S * np.exp(rate * T)
I_put = quad(lambda x: sabr_put_integ(x, S, rate, T, alpha, beta, rho, nu), 0.0, F)
I_call = quad(lambda x: sabr_call_integ(x, S, rate, T, alpha, beta, rho, nu), F, np.inf)
sabr_E_var = 2 * np.exp(rate * T) * (I_put[0] + I_call[0])
sabr_E_var

0.006350989023114547

In [18]:
print(
    f"{filename} for exotic 1 --> bs_sigma = {bs_sigma}; bach_sigma = {bach_sigma}; Black Scholes: {V_0_black_scholes:.7f}, Bachelier : {V_0_bachelier:.7f}, SABR: {V_0_SABR:.7f}"
)
print(
    f"{filename} for exotic 2 --> bs_sigma = {bs_sigma}; bach_sigma = {bach_sigma}; Black Scholes: {bs_E_var:.7f}, Bachelier : {bach_E_var:.7f}, SABR: {sabr_E_var:.7f}"
)

SPX_options for exotic 1 --> bs_sigma = 0.1849096526276905; bach_sigma = 216.60999689666605; Black Scholes: 37.7048975, Bachelier : 47.3736131, SABR: 37.7003687
SPX_options for exotic 2 --> bs_sigma = 0.1849096526276905; bach_sigma = 216.60999689666605; Black Scholes: 0.0042154, Bachelier : 11.0514293, SABR: 0.0063510
